In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 1. CUDAが利用可能かどうかを確認し、利用可能なデバイスを取得します。
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. モデルの定義
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)

# 3. データの準備
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# 4. モデルとデータをGPUに転送（すでに行っているが、念のため再確認）
model.to(device)

# 5. 損失関数と最適化アルゴリズムの定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 6. トレーニングループの作成
for epoch in range(5):  # 5エポックで訓練を行う
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # 200ミニバッチごとにロスを表示
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')


100%|██████████| 170498071/170498071 [00:12<00:00, 13875678.60it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
[1,   200] loss: 2.278
[1,   400] loss: 2.150
[1,   600] loss: 1.984
[2,   200] loss: 1.775
[2,   400] loss: 1.710
[2,   600] loss: 1.660
[3,   200] loss: 1.534
[3,   400] loss: 1.503
[3,   600] loss: 1.467
[4,   200] loss: 1.378
[4,   400] loss: 1.357
[4,   600] loss: 1.330
[5,   200] loss: 1.268
[5,   400] loss: 1.250
[5,   600] loss: 1.259
Finished Training


テンソルの中の特定の値の場所を得る

In [2]:
import torch

# 例のテンソル
tensor = torch.tensor([1, 2, 3, 4, 5, 4, 3, 2, 1])

# 特定の値
value = 4

# 特定の値のインデックスを見つける
indices = torch.where(tensor == value)
tensor = indices[0]

print(tensor)


tensor([3, 5])
